# Association Rule Mining

The problem of association rule mining is defined as:

Let ${\displaystyle I=\{i_{1},i_{2},\ldots ,i_{n}\}}$ be a set of ${\displaystyle n}$ binary attributes(usually 0 or 1 valued) called **items**.

Let ${\displaystyle D=\{t_{1},t_{2},\ldots ,t_{m}\}}$ be a set of transactions called the **database**.

Each transaction in ${\displaystyle D}$ has a unique transaction ID and contains a subset of the items in ${\displaystyle I}$.

A rule is defined as an implication of the form:

${\displaystyle X\Rightarrow Y}$, where ${\displaystyle X,Y\subseteq I}$.

<table style="float: left; margin-left: 1em; text-align:center;">
<caption>Example database with 5 transactions and 5 items</caption>
<tr>
<th>transaction ID</th>
<th>milk</th>
<th>bread</th>
<th>butter</th>
<th>beer</th>
<th>diapers</th>
</tr>
<tr>
<td>1</td>
<td>1</td>
<td>1</td>
<td>0</td>
<td>0</td>
<td>0</td>
</tr>
<tr>
<td>2</td>
<td>0</td>
<td>0</td>
<td>1</td>
<td>0</td>
<td>0</td>
</tr>
<tr>
<td>3</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>1</td>
<td>1</td>
</tr>
<tr>
<td>4</td>
<td>1</td>
<td>1</td>
<td>1</td>
<td>0</td>
<td>0</td>
</tr>
<tr>
<td>5</td>
<td>0</td>
<td>1</td>
<td>0</td>
<td>0</td>
<td>0</td>
</tr>
</table>

In [2]:
import pandas
import numpy


# please visit 'https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/' for more information
from mlxtend.preprocessing import OnehotTransactions
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


pandas.set_option('display.max_rows', 10)
pandas.set_option('display.max_columns', 10)

# set a fixed seed for numpy pseudo random generator
numpy.random.seed(100)

In [3]:
data = pandas.read_excel("./datasets/Online Retail.xlsx", 
                         parse_dates=['InvoiceDate'])

In [4]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [5]:
data.columns, data.shape

(Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
        'UnitPrice', 'CustomerID', 'Country'],
       dtype='object'), (541909, 8))

In [6]:
# tell me how many unique customers do we have?
groupby_result = data.groupby(by=["CustomerID"])

In [7]:
# users columnar count information
groupby_result.count().reset_index()

,CustomerID,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,Country
0,12346.0,2,2,2,2,2,2,2
1,12347.0,182,182,182,182,182,182,182
2,12348.0,31,31,31,31,31,31,31
3,12349.0,73,73,73,73,73,73,73
4,12350.0,17,17,17,17,17,17,17
...,...,...,...,...,...,...,...,...
4367,18280.0,10,10,10,10,10,10,10
4368,18281.0,7,7,7,7,7,7,7
4369,18282.0,13,13,13,13,13,13,13
4370,18283.0,756,756,756,756,756,756,756


In [8]:
# get the groupby result for a particular userId
groupby_result.groups[12347.0]

Int64Index([ 14938,  14939,  14940,  14941,  14942,  14943,  14944,  14945,
             14946,  14947,
            ...
            535005, 535006, 535007, 535008, 535009, 535010, 535011, 535012,
            535013, 535014],
           dtype='int64', length=182)

In [9]:
data[data.CustomerID == 12347.0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
14938,537626,85116,BLACK CANDELABRA T-LIGHT HOLDER,12,2010-12-07 14:57:00,2.10,12347.0,Iceland
14939,537626,22375,AIRLINE BAG VINTAGE JET SET BROWN,4,2010-12-07 14:57:00,4.25,12347.0,Iceland
14940,537626,71477,COLOUR GLASS. STAR T-LIGHT HOLDER,12,2010-12-07 14:57:00,3.25,12347.0,Iceland
14941,537626,22492,MINI PAINT SET VINTAGE,36,2010-12-07 14:57:00,0.65,12347.0,Iceland
14942,537626,22771,CLEAR DRAWER KNOB ACRYLIC EDWARDIAN,12,2010-12-07 14:57:00,1.25,12347.0,Iceland
...,...,...,...,...,...,...,...,...
535010,581180,20719,WOODLAND CHARLOTTE BAG,10,2011-12-07 15:52:00,0.85,12347.0,Iceland
535011,581180,21265,PINK GOOSE FEATHER TREE 60CM,12,2011-12-07 15:52:00,1.95,12347.0,Iceland
535012,581180,23271,CHRISTMAS TABLE SILVER CANDLE SPIKE,16,2011-12-07 15:52:00,0.83,12347.0,Iceland
535013,581180,23506,MINI PLAYING CARDS SPACEBOY,20,2011-12-07 15:52:00,0.42,12347.0,Iceland


In [10]:
groupby_result = data.groupby(by=["StockCode"])
product_id = groupby_result.count().reset_index()['StockCode'].astype("str")

# let's keep only products with 'StockCode' equal to 5
def is_valid_productid(x):
    
#     def is_int(x):
#         try:
#             int(x)
#         except:
#             return False
#         return True
    
#     if is_int(x):
#         return len(x) == 5
#     else:
#         return False
    return True
    
# is_valid_productid('84625A')
selcted_products = product_id[product_id.apply(is_valid_productid)]

# I
selcted_products

0              10002
1              10080
2              10120
3              10125
4              10133
            ...     
4065    gift_0001_20
4066    gift_0001_30
4067    gift_0001_40
4068    gift_0001_50
4069               m
Name: StockCode, Length: 4070, dtype: object

In [11]:
data['StockCode'] = data['StockCode'].astype("str")
raw_transactions = data[data['StockCode'].isin(selcted_products)]
raw_transactions.reset_index(inplace=True)
# raw_transactions

In [12]:
# transaction set(D)
transaction_set = []
counter = 0
for key, value in raw_transactions.groupby(by=["InvoiceNo"]):
    if counter < 10:
        print(key, len(value['StockCode']))
        counter += 1
        
    # add the data to transaction set(D)
    transaction_set.append(list(pandas.unique(value['StockCode'])))

536365 7
536366 2
536367 12
536368 4
536369 1
536370 20
536371 1
536372 2
536373 16
536374 1


In [13]:
# let's see ...
transaction_id = 0 
for transaction in transaction_set[0:5]:
    print("transaction_id = %s, items = %s\r" % 
          (transaction_id, transaction))
    transaction_id += 1

transaction_id = 0, items = ['85123A', '71053', '84406B', '84029G', '84029E', '22752', '21730']
transaction_id = 1, items = ['22633', '22632']
transaction_id = 2, items = ['84879', '22745', '22748', '22749', '22310', '84969', '22623', '22622', '21754', '21755', '21777', '48187']
transaction_id = 3, items = ['22960', '22913', '22912', '22914']
transaction_id = 4, items = ['21756']


In [14]:
# I and D
selcted_products[0:2], transaction_set[0:2]

(0    10002
 1    10080
 Name: StockCode, dtype: object,
 [['85123A', '71053', '84406B', '84029G', '84029E', '22752', '21730'],
  ['22633', '22632']])

# Additional Terminology

Let ${\displaystyle X}$ be an itemset, ${\displaystyle X\Rightarrow Y}$ an association rule and ${\displaystyle T}$ a set of transactions of a given database.

### Support
Support is an indication of how frequently the itemset appears in the dataset.

The support of ${\displaystyle X}$ with respect to ${\displaystyle T}$ is defined as the proportion of transactions ${\displaystyle t}$ in the dataset which contains the itemset ${\displaystyle X}$.

${\displaystyle \mathrm {supp} (X)={\frac {|\{t\in T;X\subseteq t\}|}{|T|}}}$


### Confidence
Confidence is an indication of how often the rule has been found to be true.

The confidence value of a rule, ${\displaystyle X\Rightarrow Y}$, with respect to a set of transactions ${\displaystyle T}$, is the proportion of the transactions that contains ${\displaystyle X}$ which also contains ${\displaystyle Y}$.

Confidence is defined as:

${\displaystyle \mathrm {conf} (X\Rightarrow Y)=\mathrm {supp} (X\cup Y)/\mathrm {supp} (X)}$

### Lift(Interest)
The lift of a rule is defined as:

${\displaystyle \mathrm {lift} (X\Rightarrow Y)={\frac {\mathrm {supp} (X\cup Y)}{\mathrm {supp} (X)\times \mathrm {supp} (Y)}}}$

or the ratio of the observed support to that expected if X and Y were independent.

### Conviction
The conviction of a rule is defined as ${\displaystyle \mathrm {conv} (X\Rightarrow Y)={\frac {1-\mathrm {supp} (Y)}{1-\mathrm {conf} (X\Rightarrow Y)}}}$.

Conviction can be interpreted as the ratio of the expected frequency that X occurs without Y (that is to say, the frequency that the rule makes an incorrect prediction) if X and Y were independent divided by the observed frequency of incorrect predictions.

### There are more!

# Association Rule Mining Process
Association rules are usually required to satisfy a user-specified **minimum support** and a user-specified **minimum confidence** at the same time. Association rule generation is usually split up into two separate steps:

* A minimum support threshold is applied to find all frequent itemsets in a database.
* A minimum confidence constraint is applied to these frequent itemsets in order to form rules.

While the second step is straightforward, the first step needs more attention.

**Brute-force search for optimal itemsets is not computationaly feasble most of the time!(power set of I has ${\displaystyle \mathrm 2^{|I|} - 1}$ members!(excluding the empty set))**

In [15]:
# let's mine!
new_transaction_set = [i for i in transaction_set if len(i) > 2]
margin = len(new_transaction_set)
oht = OnehotTransactions()
oht_ary = oht.fit_transform(new_transaction_set[0:margin])
df = pandas.DataFrame(oht_ary, columns=oht.columns_)
frequent_itemsets = apriori(df, min_support=0.04, use_colnames=True)

# show me the good stuff please!
frequent_itemsets

,support,itemsets
0,0.047234,[20712]
1,0.045496,[20719]
2,0.056572,[20724]
3,0.086487,[20725]
4,0.055812,[20726]
...,...,...
82,0.044519,[85099F]
83,0.119116,[85123A]
84,0.055486,[POST]
85,0.045225,"[22386, 85099B]"


In [16]:
# find rules with a particular 'confidence' 
association_rules(frequent_itemsets, metric="confidence", min_threshold=0.001)

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(22386),(85099B),0.066345,0.114067,0.045225,0.681669,5.976044,0.037657,2.783059
1,(85099B),(22386),0.114067,0.066345,0.045225,0.396478,5.976044,0.037657,1.547011
2,(22699),(22697),0.059232,0.056572,0.042456,0.716774,12.670108,0.039105,3.331003
3,(22697),(22699),0.056572,0.059232,0.042456,0.750480,12.670108,0.039105,3.770307


In [17]:
# find rules with a particular 'lift' 
association_rules(frequent_itemsets, metric="lift", min_threshold=1.1)

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(22386),(85099B),0.066345,0.114067,0.045225,0.681669,5.976044,0.037657,2.783059
1,(85099B),(22386),0.114067,0.066345,0.045225,0.396478,5.976044,0.037657,1.547011
2,(22699),(22697),0.059232,0.056572,0.042456,0.716774,12.670108,0.039105,3.331003
3,(22697),(22699),0.056572,0.059232,0.042456,0.750480,12.670108,0.039105,3.770307
